In [3]:
import socket, cv2, pickle, struct, threading, imutils

# Create socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host_ip = "192.168.43.153"
port = 2222
client_socket.connect((host_ip, port))

In [4]:
# Receiving Video from Client

def recv():
    data = b""
    payload_size = struct.calcsize("Q")
    while True:
        while len(data) < payload_size:
            packet = client_socket.recv(4*1024) ##### 4K
            if not packet: 
                break
            data += packet
        packed_msg_size = data[:payload_size]
        data = data[payload_size:]
        msg_size = struct.unpack("Q", packed_msg_size)[0]
            
        while len(data) < msg_size:
            data += client_socket.recv(4*1024)
        frame_data = data[:msg_size]
        data = data[msg_size:]
        frame = pickle.loads(frame_data)
        cv2.imshow("Client to server video 1", frame)
        if cv2.waitKey(1) == ord("q"):
            break
    cv2.destroyAllWindows()
    client_socket.close()

In [5]:
def sendm():
    if client_socket:
        vid = cv2.VideoCapture('http://192.168.43.1:8080/video')
        
        while (vid.isOpened()):
            try:
                img, frame = vid.read()
                frame = imutils.resize(frame, width = 620)
                a = pickle.dumps(frame)
                message = struct.pack("Q", len(a))+a
                client_socket.sendall(message)
            except ConnectionAbortedError:
                vid.release()
                client_socket.close()
                break

In [6]:
t1 = threading.Thread(target = recv)
t1.start()
t2 = threading.Thread(target = sendm)
t2.start()

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\swati\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\swati\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-292154840b42>", line 8, in recv
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\swati\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\swati\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-866de3d9b853>", line 11, in sendm
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
